In [178]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import validation_curve, learning_curve
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope
from sklearn.feature_selection import SelectKBest, f_regression




In [203]:
data = pd.read_excel("C:\\Users\\rufus\\OneDrive - The University of Manchester\\PDRA\\Sequencing\\stoke_imapact_data\\Manchester_WithMeans_250612.xlsx")
ella_data = pd.read_excel("C:\\Users\\rufus\\OneDrive - The University of Manchester\\PDRA\\Sequencing\\stoke_imapact_data\\250401_MCR_Ella.xlsx")

In [94]:
ella_data = ella_data.replace('ND',np.nan)
ella_data['IL18_V3'].astype('float64')
ella_data['PDGFBB_V4'].astype('float64')

C:\Users\rufus\AppData\Local\Temp\ipykernel_27304\4183169611.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



0      505.0
1      584.0
2      370.0
3      528.0
4        NaN
       ...  
185      NaN
186      NaN
187      NaN
188      NaN
189      NaN
Name: PDGFBB_V4, Length: 190, dtype: float64

In [204]:
#full_data = pd.merge(data, ella_data, on='Study_ID', how='inner')
data.drop(columns=['Study_ID'])

,gender 🔒,Age_years 🔒,Ethnicity 🔒,NIHSS 🔒,Date_of_Enrollment,Time_of_symptom_onset 🔒,Time_symptoms_were_recognized 🔒,Time_last_known_well_(LKW) 🔒,Date/Time_Blood_Drawn_,blooddrawdiff,...,Employment Score,Education and Skills Decile,Health and Disability Decile,Crime Decile,Barriers to Housing and Services Decile,Living Environment Decile,IDACI Rank,IDACI Decile,IDAOPI Rank,IDAOPI Decile
0,Male,58.00,White,3.000000,2021-08-05,2021-08-03 09:30:00,NaT,NaT,2021-08-06 08:30:00,71.00000,...,0.100000,5.000000,4.000000,2.000000,10.000000,3.000000,16806.00,6.000000,14026.00,5.000000
1,Male,70.00,White,2.000000,2021-08-08,2021-08-06 21:36:00,NaT,NaT,2021-08-09 08:21:00,58.75000,...,0.119000,9.000000,4.000000,2.000000,2.000000,5.000000,28304.00,9.000000,23719.00,8.000000
2,Male,59.00,BlackBritishAfricanorCarribean,4.000000,2021-08-10,NaT,2021-08-08 07:30:00,2021-08-07 23:00:00,2021-08-11 10:28:00,74.96667,...,0.185000,4.000000,2.000000,2.000000,5.000000,2.000000,4789.00,2.000000,1685.00,1.000000
3,Male,76.00,White,13.000000,2021-08-10,2021-08-09 06:45:00,NaT,NaT,2021-08-11 08:48:00,50.05000,...,0.053000,9.000000,6.000000,4.000000,10.000000,5.000000,20451.00,7.000000,15612.00,5.000000
4,Male,69.00,White,3.000000,2021-08-16,2021-08-15 07:45:00,NaT,NaT,2021-08-17 09:30:00,49.75000,...,0.235000,2.000000,1.000000,3.000000,7.000000,9.000000,5908.00,2.000000,3929.00,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,NaN,65.90,NaN,4.942105,NaT,NaT,NaT,NaT,NaT,47.15798,...,0.128441,5.129032,3.505376,4.322581,7.204301,5.543011,15087.87,5.086022,14504.37,4.903226
192,NaN,66.00,NaN,4.000000,NaT,NaT,NaT,NaT,NaT,45.81667,...,0.117500,5.000000,3.000000,4.000000,8.000000,5.000000,13651.50,5.000000,13741.50,5.000000
193,NaN,15.75,NaN,4.000000,NaT,NaT,NaT,NaT,NaT,25.87499,...,0.110250,5.000000,3.000000,5.000000,3.750000,3.750000,18508.75,6.000000,16898.25,5.000000
194,NaN,91.00,NaN,23.000000,NaT,NaT,NaT,NaT,NaT,95.33333,...,0.358000,10.000000,10.000000,10.000000,10.000000,10.000000,32513.00,10.000000,32340.00,10.000000


In [205]:
data = data.drop(range(190, 196))

In [206]:
col_to_remove = ['Date_of_Enrollment',
'Time_of_symptom_onset 🔒',
'Time_symptoms_were_recognized 🔒',
'Time_last_known_well_(LKW) 🔒',
'Date/Time_Blood_Drawn_',
'blooddrawdiff',
'MoCA_V2',
'MoCA_V3',
'MoCA_V4',
'MoCA_V5',
'6mo_imaging',
'30mo_imaging',
'Qualifying_Stroke_Date',
'V3_Date',
'V4_Date',
'V5_Date',
'V3_Post_Stroke_Days',
'V4_Post_Stroke_Days',
'V5_Post_Stroke_Days',
'TimePoint.x',
'I_feel_fatigued.x',
'I_feel_weak_all_over.x',
"I_feel_listless_('washed_out').x",
'I_feel_tired.x',
'I_have_trouble_starting_things_because_I_am_tired.x',
'I_have_trouble_finishing_things_because_I_am_tired.x',
'I_have_energy.x',
'I_am_able_to_do_my_usual_activities.x',
'I_need_to_sleep_during_the_day.x',
'I_am_too_tired_to_eat.x',
'I_need_help_doing_my_usual_activities.x',
'I_am_frustrated_by_being_too_tired_to_do_the_things_I_want_to_do.x',
'I_have_to_limit_my_social_activity_because_I_am_tired.x',
'FACIT_Fatigue_Scale_Total_(prorated,_if_answers_missing)_.x',
'Are_you_basically_satisfied_with_your_life?.x',
'Have_you_dropped_many_of_your_activities_and_interests?.x',
'Do_you_feel_that_your_life_is_empty?.x',
'Do_you_often_get_bored?.x',
'Are_you_in_good_spirits_most_of_the_time?.x',
'Are_you_afraid_that_something_bad_is_going_to_happen_to_you?.x',
'Do_you_feel_happy_most_of_the_time?.x',
'Do_you_often_feel_helpless?.x',
'Do_you_prefer_to_stay_at_home,_rather_than_going_out_and_doing_things?.x',
'Do_you_feel_that_you_have_more_problems_with_memory_than_most?.x',
'Do_you_think_it_is_wonderful_to_be_alive_now?.x',
'Do_you_feel_worthless_the_way_you_are_now?.x',
'Do_you_feel_full_of_energy?.x',
'Do_you_feel_that_your_situation_is_hopeless?.x',
'Do_you_think_that_most_people_are_better_off_than_you_are?.x',
'TimePoint.y',
'I_feel_fatigued.y',
'I_feel_weak_all_over.y',
"I_feel_listless_('washed_out').y",
'I_feel_tired.y',
'I_have_trouble_starting_things_because_I_am_tired.y',
'I_have_trouble_finishing_things_because_I_am_tired.y',
'I_have_energy.y',
'I_am_able_to_do_my_usual_activities.y',
'I_need_to_sleep_during_the_day.y',
'I_am_too_tired_to_eat.y',
'I_need_help_doing_my_usual_activities.y',
'I_am_frustrated_by_being_too_tired_to_do_the_things_I_want_to_do.y',
'I_have_to_limit_my_social_activity_because_I_am_tired.y',
'FACIT_Fatigue_Scale_Total_(prorated,_if_answers_missing)_.y',
'Are_you_basically_satisfied_with_your_life?.y',
'Have_you_dropped_many_of_your_activities_and_interests?.y',
'Do_you_feel_that_your_life_is_empty?.y',
'Do_you_often_get_bored?.y',
'Are_you_in_good_spirits_most_of_the_time?.y',
'Are_you_afraid_that_something_bad_is_going_to_happen_to_you?.y',
'Do_you_feel_happy_most_of_the_time?.y',
'Do_you_often_feel_helpless?.y',
'Do_you_prefer_to_stay_at_home,_rather_than_going_out_and_doing_things?.y',
'Do_you_feel_that_you_have_more_problems_with_memory_than_most?.y',
'Do_you_think_it_is_wonderful_to_be_alive_now?.y',
'Do_you_feel_worthless_the_way_you_are_now?.y',
'Do_you_feel_full_of_energy?.y',
'Do_you_feel_that_your_situation_is_hopeless?.y',
'Do_you_think_that_most_people_are_better_off_than_you_are?.y',
'TimePoint',
'I_feel_fatigued',
'I_feel_weak_all_over',
"I_feel_listless_('washed_out')",
'I_feel_tired',
'I_have_trouble_starting_things_because_I_am_tired',
'I_have_trouble_finishing_things_because_I_am_tired',
'I_have_energy',
'I_am_able_to_do_my_usual_activities',
'I_need_to_sleep_during_the_day',
'I_am_too_tired_to_eat',
'I_need_help_doing_my_usual_activities',
'I_am_frustrated_by_being_too_tired_to_do_the_things_I_want_to_do',
'I_have_to_limit_my_social_activity_because_I_am_tired',
'FACIT_Fatigue_Scale_Total_(prorated,_if_answers_missing)_',
'Are_you_basically_satisfied_with_your_life?',
'Have_you_dropped_many_of_your_activities_and_interests?',
'Do_you_feel_that_your_life_is_empty?',
'Do_you_often_get_bored?',
'Are_you_in_good_spirits_most_of_the_time?',
'Are_you_afraid_that_something_bad_is_going_to_happen_to_you?',
'Do_you_feel_happy_most_of_the_time?',
'Do_you_often_feel_helpless?',
'Do_you_prefer_to_stay_at_home,_rather_than_going_out_and_doing_things?',
'Do_you_feel_that_you_have_more_problems_with_memory_than_most?',
'Do_you_think_it_is_wonderful_to_be_alive_now?',
'Do_you_feel_worthless_the_way_you_are_now?',
'Do_you_feel_full_of_energy?',
'Do_you_feel_that_your_situation_is_hopeless?',
'Do_you_think_that_most_people_are_better_off_than_you_are?',
'Is_ECG_available? 🔒',
'Are_labs_available? 🔒',
'Direct_LDL',
'LangFact_V3',
'WMF_V3',
'ProcSpeed_V3',
'MemFact_V3',
'Visuospatial_V3',
'GlobalTest_V3',
'GlobalDomain_V3',
'Impaired_at_V3',
'LangFact_V4',
'WMF_V4',
'ProcSpeed_V4',
'MemFact_V4',
'Visuospatial_V4',
'GlobalTest_V4',
'GlobalDomain_V4',
'Impaired_at_V4',
'LangFact_V5',
'WMF_V5',
'ProcSpeed_V5',
'MemFact_V5',
'Visuospatial_V5',
'GlobalTest_V5',
'GlobalDomain_V5',
'Impaired_at_V5',
'crp 🔒',
'no_med_classes_taken 🔒',
'Have Antibiotics Commenced?',
'Reason for Antibiotics',
'If other, specify',
'Index of Multiple Deprivation Decile',
'Income Decile',
'Income Score',
'Employment Decile',
'Employment Score',
'Education and Skills Decile',
'Health and Disability Decile',
'Crime Decile',
'Barriers to Housing and Services Decile',
'Living Environment Decile',
'Date_of_Enrollment',
'Time_symptoms_were_recognized 🔒',
'Time_last_known_well_(LKW) 🔒',
'Date/Time_Blood_Drawn_',
'Previous Stroke 🔒',
'Normed_MoCA_V3',
'Normed_MoCA_V4',
'Normed_MoCA_V5']


data = data.drop(columns=col_to_remove)

In [164]:
data.to_csv('data.csv')

In [207]:
data['Normed_MoCA_V2'].isna().sum()

np.int64(57)

In [214]:
data

,Study_ID,gender 🔒,Age_years 🔒,Ethnicity 🔒,NIHSS 🔒,Lesion_Location,Lacunar,IV_Thrombolysis 🔒,IA_Thrombectomy 🔒,stroke_AMPM 🔒,...,ACE_Inhibitors 🔒,Beta_Blockers 🔒,ARBs 🔒,CCBs 🔒,Other_Antihypertensives 🔒,Antidepressants 🔒,IDACI Rank,IDACI Decile,IDAOPI Rank,IDAOPI Decile
0,4133-001,Male,58.0,White,3.0,Right,Yes,No,No,AM,...,0.0,0.0,1.0,1.0,0.0,0.0,16806.0,6.0,14026.0,5.0
1,4133-002,Male,70.0,White,2.0,Left,Yes,No,No,PM,...,1.0,0.0,0.0,1.0,0.0,0.0,28304.0,9.0,23719.0,8.0
2,4133-003,Male,59.0,BlackBritishAfricanorCarribean,4.0,Right,Yes,No,No,AM,...,1.0,0.0,0.0,1.0,1.0,0.0,4789.0,2.0,1685.0,1.0
3,4133-004,Male,76.0,White,13.0,Right,No,No,No,AM,...,0.0,0.0,0.0,0.0,0.0,0.0,20451.0,7.0,15612.0,5.0
4,4133-005,Male,69.0,White,3.0,Left,Yes,Yes,No,AM,...,0.0,1.0,0.0,0.0,0.0,0.0,5908.0,2.0,3929.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,4133-200,Male,85.0,White,4.0,Right,No,No,No,PM,...,0.0,0.0,0.0,0.0,0.0,0.0,18213.0,6.0,20096.0,7.0
183,4133-201,Female,79.0,White,4.0,Left,Yes,No,No,AM,...,0.0,0.0,0.0,0.0,0.0,0.0,2435.0,1.0,3143.0,1.0
184,4133-202,Female,52.0,White,9.0,Right,No,No,Yes,AM,...,0.0,0.0,0.0,0.0,0.0,1.0,16420.0,5.0,19588.0,6.0
185,4133-203,Male,58.0,White,1.0,Bilateral,No,No,No,PM,...,0.0,1.0,1.0,0.0,0.0,0.0,3329.0,2.0,5799.0,2.0


In [212]:
print(data['Normed_MoCA_V2'])
list(data.dtypes)
data = data.dropna(subset=['Normed_MoCA_V2'])

0      25.0
1      26.0
2      20.0
3      29.0
4      28.0
       ... 
185    27.0
186    28.0
187     NaN
188     NaN
189     NaN
Name: Normed_MoCA_V2, Length: 190, dtype: float64


In [215]:
categorical_cols = data.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in categorical_cols:
    data[col] = le.fit_transform(data[col].astype(str)) 

C:\Users\rufus\AppData\Local\Temp\ipykernel_27304\229524314.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\rufus\AppData\Local\Temp\ipykernel_27304\229524314.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\rufus\AppData\Local\Temp\ipykernel_27304\229524314.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [216]:
print("Category Mapping:", le.classes_)

Category Mapping: ['Frequent' 'Infrequent' 'Unknown' 'nan']


In [217]:
X = data.drop(columns='Normed_MoCA_V2')
y = data['Normed_MoCA_V2']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [218]:
#Define the space over which hyperopt will search for optimal hyperparameters.
space = {
    'max_depth': scope.int(hp.quniform("max_depth", 1, 5, 1)),
    'gamma': hp.uniform ('gamma', 0,1),
    'reg_alpha' : hp.uniform('reg_alpha', 0,50),
    'reg_lambda' : hp.uniform('reg_lambda', 10,100),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0,1),
    'min_child_weight' : hp.uniform('min_child_weight', 0,5),
    'learning_rate': hp.uniform('learning_rate', 0, .3),
    'max_bin' : scope.int(hp.quniform('max_bin', 200, 550, 1)),
    'n_estimators': 10000,
    'random_state': 42,
    'eval_metric' : 'rmse',
    'early_stopping_rounds' : 1000
    }

In [219]:
#Define the hyperopt objective.
def hyperparameter_tuning(space):
    model = XGBRegressor(**space)
    
    #Define evaluation datasets.
    evaluation = [(X_train, y_train), (X_val, y_val)]
    
    #Fit the model.
    model.fit(X_train, y_train,
            eval_set=evaluation,
            verbose=False)

    #Obtain prediction and rmse score.
    preds = model.predict(X_val)
    rmse = mean_squared_error(y_val, preds)
    r2 = r2_score(y_val, preds)
    print ("SCORE:", rmse, r2)
    
    #Specify what the loss is for each model.
    return {'loss':-r2, 'status': STATUS_OK, 'model': model}

In [ ]:
#Run 30 trials.
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print(best)

SCORE:                                                 
9.784640640751109                                      
-0.15057484696878887                                   
SCORE:                                                                            
9.96550881034504                                                                  
-0.17184311569654964                                                              
SCORE:                                                                            
9.627522063744022                                                                 
-0.1320992902944862                                                               
SCORE:                                                                            
9.970237842597932                                                                
-0.17239920209363402                                                             
SCORE:                                                                           
9.965

In [225]:
best_converted = {key: int(value) if key in ['max_depth', 'max_bin'] else value for key, value in best.items()}
#These fixed params dont come through via fmin so need to be applied to best 
best_converted.update({
    'eval_metric': 'rmse',
    'early_stopping_rounds': 1000,
    'n_estimators': 1000,
    'random_state': 5
})

In [226]:
bst = XGBRegressor(**best_converted)

# Train 
bst.fit(X_train, y_train, 
        eval_set=[(X_val, y_val)], 
        verbose=False)

# predictions
preds = bst.predict(X_val)

importance = bst.feature_importances_
feature_names = X_train.columns

feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values('Importance', ascending=False)

mse = mean_squared_error(y_val, preds)
r2 = r2_score(y_val, preds)
mae = np.mean(np.abs(y_val - preds))

print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.3f}")
print(f"Mean Absolute Error: {mae:.2f}")

Mean Squared Error: 9.97
R² Score: -0.172
Mean Absolute Error: 2.52


In [227]:
bst = XGBRegressor(**best_converted)

# Train 
bst.fit(X_train, y_train, 
        eval_set=[(X_val, y_val)], 
        verbose=False)

# predictions
preds = bst.predict(X_test)

importance = bst.feature_importances_
feature_names = X_train.columns

feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values('Importance', ascending=False)

mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
mae = np.mean(np.abs(y_test - preds))

print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.3f}")
print(f"Mean Absolute Error: {mae:.2f}")

Mean Squared Error: 14.78
R² Score: 0.018
Mean Absolute Error: 2.99


In [228]:
print(f"Train size: {X_train.shape[0]}")
print(f"Val size: {X_val.shape[0]}")  
print(f"Test size: {X_test.shape[0]}")
print(f"Total features: {X_train.shape[1]}")

Train size: 84
Val size: 22
Test size: 27
Total features: 47


In [229]:
print("Top 10 Most Important Features:")
print(feature_importance.head(10))

# Get predictions for the TEST set (not validation set)
test_preds = bst.predict(X_test)

# And see which patients were predicted
results = pd.DataFrame({
    'Row_Index': X_test.index,
    'Actual_MoCA': y_test.values,
    'Predicted_MoCA': test_preds,  # Use test predictions
    'Error': y_test.values - test_preds  # Use test predictions for error
})
print(results.head())

print("Top 10 Worst Important Features:")
print(feature_importance.tail(10))

Top 10 Most Important Features:
                        Feature  Importance
12  Atrial_Fibrillation_(Afib)?    0.114541
14                          HDL    0.079027
15                        HbA1C    0.073882
29           Alcohol_Consumed 🔒    0.071380
7             IV_Thrombolysis 🔒    0.046974
16   White_Cell_Count_(109/l) 🔒    0.045668
34              Antiplatelets 🔒    0.045417
10         stroke_time_of_day 🔒    0.043743
43                   IDACI Rank    0.040323
17        Neutrophils_(109/l) 🔒    0.039022
   Row_Index  Actual_MoCA  Predicted_MoCA     Error
0         62         29.0       24.391066  4.608934
1         91         27.0       26.165897  0.834103
2         38         24.0       26.165897 -2.165897
3         51         27.0       24.391066  2.608934
4        161         16.0       24.595741 -8.595741
Top 10 Worst Important Features:
                      Feature  Importance
21        Basophils_(109/l) 🔒         0.0
26           Hyperlipidemia 🔒         0.0
38           